## Compute the Global Feature Rankings 

In this notebook, we compute the global feature rankings. The methods include:
1. PD and ALE variance (PAPER)
2. Backward Single-pass Permutation Importance 
3. Forward Single-pass Permutation Importance 
4. Backward Multi-pass Permutation Importance  
5. Forward Multi-pass Permutation Importance 
6. Random Forest Gini Impurity 
7. Logistic Regression Coefficients 
8. Summed SHAP values 
9. SAGE values 

In [23]:
import sys, os 
from os.path import dirname
path = dirname(dirname(os.getcwd()))
sys.path.insert(0, path)
sys.path.insert(0, '/home/monte.flora/python_packages/scikit-explain')

In [20]:
import skexplain 
from skexplain.common.importance_utils import to_skexplain_importance
from src.io.io import load_data_and_model
from src.common.util import subsampler, normalize_importance, compute_sage

import pickle
import shap
import itertools

In [3]:
# Constants. 
N_BOOTSTRAP = 1
N_BINS = 20 
N_JOBS = 30 
N_PERMUTE = 5
SIZE = 5000
EVALUATION_FN = 'norm_aupdc'
RESULTS_PATH = os.path.join(path, 'results')
BASE_PATH = '/work/mflora/explainability_work/'
DATA_BASE_PATH = os.path.join(BASE_PATH, 'datasets')
MODEL_BASE_PATH = os.path.join(BASE_PATH, 'models')

DATASETS = ['tornado', 'severe_wind', 'severe_hail', 'road_surface']
OPTIONS = ['original', 'reduced']

In [4]:
# Load the Data and Model 
dataset = 'tornado'
option = 'reduced'
model, X, y = load_data_and_model(dataset, option, DATA_BASE_PATH, MODEL_BASE_PATH)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [5]:
# Subsample the dataset. 
X_sub, y_sub = subsampler(X,y,SIZE)
features = list(X.columns)
est_name = model[0]

In [7]:
explainer = skexplain.ExplainToolkit(model, X_sub, y_sub)

### 1. ALE variance 

In [6]:
# Compute ALE 
ale = explainer.ale(features='all', n_bootstrap=N_BOOTSTRAP, n_bins=N_BINS)
# Compute the variance. 
ale_var = explainer.ale_variance(ale)
# Convert to feature rankings 
ale_rank = to_skexplain_importance(ale_var[f'ale_variance_scores__{est_name}'].values, 
                                          est_name, features, method='ale_variance', normalize=True)

# Save the raw ALE and ALE-variance rankings results for paper 1 
explainer.save(os.path.join(RESULTS_PATH, f'ale_{dataset}_{option}.nc'), ale)
explainer.save(os.path.join(RESULTS_PATH, f'ale_rank_{dataset}_{option}.nc'), ale_rank)

ALE Numerical Features:   0%|          | 0/14 [00:00<?, ?it/s]

## 2-5. Flavors of Permutation Importance 

In [7]:
# Compute the permutatation importance (forward, backward, single-pass, multi-pass)
explainer = skexplain.ExplainToolkit(model, X_sub, y_sub)
DIRECTIONS = ['forward', 'backward']
n_vars = len(X.columns)

for direction in DIRECTIONS: 
    results = explainer.permutation_importance(
                                           n_vars=n_vars, 
                                           evaluation_fn=EVALUATION_FN,
                                           n_permute=N_PERMUTE, 
                                           n_jobs=N_JOBS,
                                           verbose=True,
                                           random_seed=42, 
                                           direction=direction,
                                              )
    # The native output for skexplain (based on PermutationImportance) are the permuted scores
    # (i.e., the scores after features are permuted). However, for this study, we need to convert 
    # the output to a proper importance scores (i.e., with respect to the original score such that higher 
    # values indicate higher importances). Though proper importance scores are often shown as ratios, we found 
    # the difference (e.g., original score - permuted score) provides better scores. Ratio data does not have 
    # favorable properties for importance scores. Additionally, we normalize the importance scores 
    # for comparison across the different methods. 
    est_name = model[0]
    if direction == 'backward':
        # Backward multipass: 1 + (permute score - original score)
        # Backward singlepass: original_score - permuted score
        original_score = results[f'original_score__{est_name}'].values
        scores = 1.0 + (results[f'backward_multipass_scores__{est_name}'].values - original_score)
        norm_scores = normalize_importance(scores)
        results[f'backward_multipass_scores__{est_name}'] = (['n_vars_multipass', 'n_permute'], norm_scores)
    
        original_score = results[f'original_score__{est_name}'].values
        scores = original_score - results[f'backward_singlepass_scores__{est_name}'].values
        norm_scores = normalize_importance(scores)
        results[f'backward_singlepass_scores__{est_name}'] = (['n_vars_singlepass', 'n_permute'], norm_scores)
    
    else: 
        # Forward Multiplass: original_score - permuted score
        # Forward Singlepass: 1 + (permute score - original score)
        original_score = results[f'original_score__{est_name}'].values
        scores = 1.0 + (results[f'forward_singlepass_scores__{est_name}'].values - original_score)
        norm_scores = normalize_importance(scores)
        results[f'forward_singlepass_scores__{est_name}'] = (['n_vars_singlepass', 'n_permute'], norm_scores)
        
        original_score = results[f'original_score__{est_name}'].values
        scores = original_score - results[f'forward_multipass_scores__{est_name}'].values
        norm_scores = normalize_importance(scores)
        results[f'forward_multipass_scores__{est_name}'] = (['n_vars_multipass', 'n_permute'], norm_scores)
        
    # Save the results 
    explainer.save(os.path.join(RESULTS_PATH, f'perm_imp_rank_{direction}_{dataset}_{option}.nc'), results)


Perm. Imp.: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:19<00:00,  1.39s/it]

Perm. Imp.: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:19<00:00,  1.36s/it]


## 4. Partial Dependence 

In [10]:
# Compute PD 
pd = explainer.pd(features='all', n_bootstrap=N_BOOTSTRAP, n_bins=N_BINS)
# Compute the variance. 
pd_var = explainer.pd_variance(pd)
# Convert to feature rankings 
pd_rank = to_skexplain_importance(pd_var[f'pd_variance_scores__{est_name}'].values, 
                                           est_name, features, method='pd_variance', normalize=True)

# Save the raw PD and PD-variance rankings results for paper 1 
explainer.save(os.path.join(RESULTS_PATH, f'pd_{dataset}_{option}.nc'), pd)
explainer.save(os.path.join(RESULTS_PATH, f'pd_rank_{dataset}_{option}.nc'), pd_rank)

PD Numerical Features:   0%|          | 0/14 [00:00<?, ?it/s]

## 5. Model-Specific (Gini Impurity and Regression Coefficients)

In [11]:
# Model-specific 
# Get the logistic regression coefficients
if dataset == 'road_surface':
    if option == 'original':
        gini_values = model[1].base_estimator_.feature_importances_
    else:
        gini_values = model[1].base_estimator.named_steps['model'].feature_importances_
    
    gini_rank = to_skexplain_importance(gini_values,
                                       estimator_name='Random Forest', 
                                       feature_names=features, 
                                         method = 'gini')
    
    # Save the random forest importances.
    explainer.save(os.path.join(RESULTS_PATH, f'gini_rank_{dataset}_{option}.nc'), gini_rank)
    
else:    
    coefs = model[1].base_estimator.named_steps['model'].coef_[0]
    coef_rank = to_skexplain_importance(coefs,
                                       estimator_name=est_name, 
                                       feature_names=features, 
                                       method = 'coefs')
    
    # Save the log. regress. importances.
    explainer.save(os.path.join(RESULTS_PATH, f'coef_rank_{dataset}_{option}.nc'), coef_rank)

## 6. SHAP Values

In [18]:
# Compute SHAP (Approx. Owen Values)
# The default explainer is the PermutationExplainer. The PermutationExplainer uses a 
# simple forward- and backward-permutation scheme to compute the SHAP values. 
# The SHAP documentation claims this method is exact for 2nd order interactions.
# For the maskers, we are using correlations and as such we are computing 
# approximate Owen values. 

# Check if each SHAP example can be ran in parallel. 
results = explainer.local_attributions('shap', 
                                       shap_kws={'masker' : 
                                      shap.maskers.Partition(X, max_samples=100, 
                                                             clustering="correlation"), 
                                     'algorithm' : 'permutation'})


shap_rank = to_skexplain_importance(results[f'shap_values__{est_name}'].values, 
                                     estimator_name=est_name, 
                                     feature_names=features, 
                                     method ='shap_sum')

# Sum the SHAP values for each feature and then save results. 
explainer.save(os.path.join(RESULTS_PATH, f'shap_{dataset}_{option}.nc'), results)
explainer.save(os.path.join(RESULTS_PATH, f'shap_rank_{dataset}_{option}.nc'), shap_rank)

## 7. SAGE Values

In [22]:
# Compute SAGE
sage_values = compute_sage(model[1], X_sub.values, y_sub, X)
sage_rank = to_skexplain_importance(sage_values,
                                     estimator_name=est_name, 
                                     feature_names=features, 
                                     method = 'sage')

# Sum the SAGE values for each feature and then save results. 
explainer.save(os.path.join(RESULTS_PATH, f'sage_rank_{dataset}_{option}.nc'), sage_rank)

## 8. Tree Interpreter

In [ ]:
if dataset == 'road_surface':
    results = explainer.local_attributions('tree_interpreter')

    ti_rank = to_skexplain_importance(results[f'tree_interpreter_values__{est_name}'].values, 
                                     estimator_name=est_name, 
                                     feature_names=features, 
                                     method ='tree_interpreter')

    # Sum the SHAP values for each feature and then save results. 
    explainer.save(os.path.join(RESULTS_PATH, f'ti_{dataset}_{option}.nc'), results)
    explainer.save(os.path.join(RESULTS_PATH, f'ti_rank_{dataset}_{option}.nc'), ti_rank)

## 9. LIME

In [ ]:
# For the LIME, we must provide the training dataset. We also denote any categorical features. 
if dataset == 'road_surface':
    lime_kws = {'training_data' : X.values, 'categorical_names' : ['rural', 'urban']}
else:
    lime_kws = {'training_data' : X.values}

results = explainer.local_attributions('lime', lime_kws=lime_kws)

lime_rank = to_skexplain_importance(results[f'lime_values__{est_name}'].values, 
                                     estimator_name=est_name, 
                                     feature_names=features, 
                                     method ='lime')

# Sum the SHAP values for each feature and then save results. 
explainer.save(os.path.join(RESULTS_PATH, f'lime_{dataset}_{option}.nc'), results)
explainer.save(os.path.join(RESULTS_PATH, f'lime_rank_{dataset}_{option}.nc'), lime_rank)